In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import xml.etree.ElementTree as ET
from google.colab import drive
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image
import pandas as pd

In [ ]:
!gdown --id 1pe4_HgXb9dctFGJXVNlyNcKSXZeht0lX

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1pe4_HgXb9dctFGJXVNlyNcKSXZeht0lX
From (redirected): https://drive.google.com/uc?id=1pe4_HgXb9dctFGJXVNlyNcKSXZeht0lX&confirm=t&uuid=543d8409-41f7-4a64-a339-58fec68bcd88
To: /content/car_img-test.zip
100% 2.51G/2.51G [00:55<00:00, 45.4MB/s]


In [ ]:
!unzip /content/car_img-test.zip -d /content/drive/MyDrive/project2/

Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/MyDrive/project2/test/day_08471.jpg  
  inflating: /content/drive/MyDrive/project2/test/day_08471.xml  
  inflating: /content/drive/MyDrive/project2/test/day_08473.jpg  
  inflating: /content/drive/MyDrive/project2/test/day_08473.xml  
  inflating: /content/drive/MyDrive/project2/test/day_08475.jpg  
  inflating: /content/drive/MyDrive/project2/test/day_08475.xml  
  inflating: /content/drive/MyDrive/project2/test/day_08477.jpg  
  inflating: /content/drive/MyDrive/project2/test/day_08477.xml  
  inflating: /content/drive/MyDrive/project2/test/day_08478.jpg  
  inflating: /content/drive/MyDrive/project2/test/day_08478.xml  
  inflating: /content/drive/MyDrive/project2/test/day_08487.jpg  
  inflating: /content/drive/MyDrive/project2/test/day_08487.xml  
  inflating: /content/drive/MyDrive/project2/test/day_08493.jpg  
  inflating: /content/drive/MyDrive/project2/test/day_08493.xml  
  inflating: /content/dri

In [ ]:
!gdown --id 1hwz6X-Zp7JpJL35K6P3z7k6O_PTXhUcT

In [ ]:
!unzip /content/car_img-validation.zip -d /content/drive/MyDrive/project2/

Archive:  /content/car_img-validation.zip
   creating: /content/drive/MyDrive/project2/validation/
  inflating: /content/drive/MyDrive/project2/validation/day_00004.jpg  
  inflating: /content/drive/MyDrive/project2/validation/day_00004.xml  
  inflating: /content/drive/MyDrive/project2/validation/day_00011.jpg  
  inflating: /content/drive/MyDrive/project2/validation/day_00011.xml  
  inflating: /content/drive/MyDrive/project2/validation/day_00018.jpg  
  inflating: /content/drive/MyDrive/project2/validation/day_00018.xml  
  inflating: /content/drive/MyDrive/project2/validation/day_00024.jpg  
  inflating: /content/drive/MyDrive/project2/validation/day_00024.xml  
  inflating: /content/drive/MyDrive/project2/validation/day_00033.jpg  
  inflating: /content/drive/MyDrive/project2/validation/day_00033.xml  
  inflating: /content/drive/MyDrive/project2/validation/day_00036.jpg  
  inflating: /content/drive/MyDrive/project2/validation/day_00036.xml  
  inflating: /content/drive/MyDrive/p

In [ ]:
import shutil

root_dir = '/content/drive/MyDrive/project2/test'

xml_dir = os.path.join(root_dir, 'xml_test')
jpg_dir = os.path.join(root_dir, 'jpg_test')

os.makedirs(xml_dir, exist_ok=True)
os.makedirs(jpg_dir, exist_ok=True)

for filename in os.listdir(root_dir):
    if filename.endswith('.xml'):
        shutil.move(os.path.join(root_dir, filename), xml_dir)
    elif filename.endswith('.jpg'):
        shutil.move(os.path.join(root_dir, filename), jpg_dir)

In [ ]:
xml_dir = '/content/drive/MyDrive/project2/test/xml_test'

yolo_dir = '/content/drive/MyDrive/project2/test_yolo'

if not os.path.exists(yolo_dir):
    os.makedirs(yolo_dir)

classes = set()

for filename in os.listdir(xml_dir):
    if filename.endswith(".xml"):
        tree = ET.parse(os.path.join(xml_dir, filename))
        root = tree.getroot()
        for obj in root.findall('object'):
            obj_name = obj.find('name').text
            if obj_name != 'کل ناحیه پلاک':
                classes.add(obj_name)

for filename in os.listdir(xml_dir):
    if filename.endswith(".xml"):
        tree = ET.parse(os.path.join(xml_dir, filename))
        root = tree.getroot()

        image_name = root.find('filename').text

        yolo_file = open(os.path.join(yolo_dir, image_name.replace('.jpg', '.txt')), 'w')

        for obj in root.findall('object'):
            obj_name = obj.find('name').text
            if obj_name != 'کل ناحیه پلاک':
                bndbox = obj.find('bndbox')
                xmin = float(bndbox.find('xmin').text)
                ymin = float(bndbox.find('ymin').text)
                xmax = float(bndbox.find('xmax').text)
                ymax = float(bndbox.find('ymax').text)

                x_center = (xmin + xmax) / 2
                y_center = (ymin + ymax) / 2
                width = xmax - xmin
                height = ymax - ymin

                class_id = list(classes).index(obj_name)
                yolo_file.write(f'{class_id} {x_center} {y_center} {width} {height}\n')

        yolo_file.close()

In [22]:
!pip install ultralytics
from ultralytics import YOLO
model = YOLO("yolov8n.yaml")

In [23]:
import torch
yolo_dir = '/content/drive/MyDrive/project2/test_yolo'

model_path = 'yolov8n.pt'

classes = set()

for filename in os.listdir(xml_dir):
    if filename.endswith(".xml"):
        tree = ET.parse(os.path.join(xml_dir, filename))
        root = tree.getroot()
        for obj in root.findall('object'):
            obj_name = obj.find('name').text
            if obj_name != 'کل ناحیه پلاک':
                classes.add(obj_name)

classes = list(classes)

for filename in os.listdir(yolo_dir):
    if filename.endswith(".txt"):
        image_name = filename.replace('.txt', '.jpg')
        image_path = os.path.join('/content/drive/MyDrive/project2/test', image_name)
        image = cv2.imread(image_path)

        results = model(image)

results_df = pd.DataFrame(results.pandas().xyxy[0])

for index, row in results_df.iterrows():
    x1, y1, x2, y2, confidence, class_id = row
    class_name = classes[int(class_id)]
    cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
    cv2.putText(image, f'{class_name} {confidence:.2f}', (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

WARNING ⚠️ 'source' is missing. Using 'source=/usr/local/lib/python3.10/dist-packages/ultralytics/assets'.

image 1/2 /usr/local/lib/python3.10/dist-packages/ultralytics/assets/bus.jpg: 640x480 (no detections), 300.3ms
image 2/2 /usr/local/lib/python3.10/dist-packages/ultralytics/assets/zidane.jpg: 384x640 (no detections), 241.1ms
Speed: 4.1ms preprocess, 270.7ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
WARNING ⚠️ 'source' is missing. Using 'source=/usr/local/lib/python3.10/dist-packages/ultralytics/assets'.

image 1/2 /usr/local/lib/python3.10/dist-packages/ultralytics/assets/bus.jpg: 640x480 (no detections), 296.9ms
image 2/2 /usr/local/lib/python3.10/dist-packages/ultralytics/assets/zidane.jpg: 384x640 (no detections), 226.2ms
Speed: 7.9ms preprocess, 261.6ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
WARNING ⚠️ 'source' is missing. Using 'source=/usr/local/lib/python3.10/dist-packages/ultralytics/assets'.

image 1/2 /usr/local/lib/pytho

KeyboardInterrupt: 